<a href="https://colab.research.google.com/github/dickyprasetiyo/afas/blob/master/Salinan_dari_Task_Clusification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
data_original = pd.read_csv("air_bnb.csv")

In [0]:
data_original.isnull().sum() 
#Lihat apakah ada nilai null pada dataset ternyata ada
# Pada baris name,host_name,last_review, dan reviews_per_month
# Saya akan menghapus baris yang terdapat nilai null

id                                   0
name                                59
host_id                              0
host_name                           26
neighbourhood_group                  0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       3908
reviews_per_month                 3914
calculated_host_listings_count       0
availability_365                     0
dtype: int64

In [0]:
data_berubah1 = data_original.dropna(axis=1)

In [0]:
kualitatif = {"neighbourhood_group":{"Charlottenburg-Wilm.":0,"Friedrichshain-Kreuzberg":1,"Lichtenberg":2,"Marzahn - Hellersdorf":3,
                                    "Mitte":4,"Neukölln":5,"Pankow":6,"Reinickendorf":7,"Spandau":8,"Steglitz - Zehlendorf":9,
                                     "Steglitz - Zehlendorf":10,"Tempelhof - Schöneberg":11,"Treptow - Köpenick":12},
             "room_type":{"Entire home/apt":0,"Private room":1,"Shared room":2}}

In [0]:
data_berubah1.replace(kualitatif, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6666: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regex=regex,


In [0]:
data_berubah2 = data_berubah1.drop("neighbourhood", axis=1)

In [0]:
Q1 = data_berubah2['price'].quantile(0.25)
Q3 = data_berubah2['price'].quantile(0.75)
IQR = Q3 - Q1    #IQR is interquartile range. 

filter1 = (data_berubah2['price'] >= Q1 - 1.5 * IQR) & (data_berubah2['price'] <= Q3 + 1.5 *IQR)
data_berubah3 = data_berubah2.loc[filter1]  

In [0]:
Q1a = data_berubah3['minimum_nights'].quantile(0.25)
Q3a = data_berubah3['minimum_nights'].quantile(0.75)
IQRa = Q3a - Q1a    #IQR is interquartile range. 

filter1a = (data_berubah3['minimum_nights'] >= Q1a - 1.5 * IQRa) & (data_berubah3['minimum_nights'] <= Q3a + 1.5 *IQRa)
data_berubah4 = data_berubah3.loc[filter1a]  

In [0]:
data_berubah4

,id,host_id,neighbourhood_group,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,calculated_host_listings_count,availability_365
0,2015,2217,4,52.534537,13.402557,0,60,4,118,4,141
1,2695,2986,6,52.548513,13.404553,1,17,2,6,1,0
3,3309,4108,11,52.498855,13.349065,1,26,5,25,1,297
4,7071,17391,6,52.543157,13.415091,1,42,2,197,1,26
9,17409,67590,6,52.529071,13.412843,1,45,3,279,1,42
...,...,...,...,...,...,...,...,...,...,...,...
22547,29856708,87555909,4,52.533865,13.400731,0,60,2,0,1,314
22548,29857108,67537363,11,52.496211,13.341738,2,20,1,0,6,78
22549,29864272,3146923,6,52.531800,13.411999,0,85,3,0,2,15
22550,29866805,36961901,4,52.520802,13.378688,1,99,1,0,3,6


In [0]:
X = data_berubah4.drop(["latitude","longitude","id","host_id","availability_365"], axis=1)
y = data_berubah4['room_type']

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [0]:
from sklearn.svm import SVC
svclassifier_1 = SVC(kernel='rbf')
svclassifier_1.fit(X_train, y_train)
svclassifier_2 = SVC(kernel='linear')
svclassifier_2.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
y_pred1 = svclassifier_1.predict(X_test)
y_pred2 = svclassifier_2.predict(X_test)

In [0]:
y_pred1

array([1, 1, 1, ..., 1, 1, 1])

In [0]:
y_pred2

array([1, 1, 1, ..., 1, 1, 1])

In [0]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred1))
print(classification_report(y_test,y_pred1))

[[1606   33    0]
 [  70 2031    0]
 [   1   40    4]]
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1639
           1       0.97      0.97      0.97      2101
           2       1.00      0.09      0.16        45

    accuracy                           0.96      3785
   macro avg       0.97      0.68      0.70      3785
weighted avg       0.96      0.96      0.96      3785



In [0]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test,y_pred2))
print(classification_report(y_test,y_pred2))

[[1639    0    0]
 [   0 2101    0]
 [   0    0   45]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1639
           1       1.00      1.00      1.00      2101
           2       1.00      1.00      1.00        45

    accuracy                           1.00      3785
   macro avg       1.00      1.00      1.00      3785
weighted avg       1.00      1.00      1.00      3785

